<a href="https://colab.research.google.com/github/InduriRam/Cryptocurrency-price-prediction-using-RNN-/blob/master/crypt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded1 = files.upload()

Saving BCH-USD.csv to BCH-USD.csv


In [3]:
uploaded2=files.upload()

Saving BTC-USD.csv to BTC-USD.csv


In [5]:
uploaded3 = files.upload()

Saving ETH-USD.csv to ETH-USD.csv


In [7]:
uploaded4 = files.upload()

Saving LTC-USD.csv to LTC-USD.csv


In [16]:
import pandas as pd
import os
from sklearn.preprocessing import StandardScaler
from collections import  deque
import sklearn
import  random
import numpy as np
import time
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,LSTM,BatchNormalization,CuDNNLSTM
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
SEQ_LEN = 60 #last 60 min of pricing used to make prediction
FUTURE_PERIOD_PREDICT = 3 #prediction to 3 min into future
RATIO_TO_PREDICT = "BTC-USD" #change this to  predict for different crypto currencies
EPOCHS = 9
BATCH_SIZE = 64
NAME = f"{SEQ_LEN}-SEQ-{FUTURE_PERIOD_PREDICT}-PRED-{int(time.time())}"
def preprocess_df(df):
    df = df.drop('future',1)

    for col in df.columns:
        if col != "target":
            df[col] = df[col].pct_change()
            df.dropna(inplace = True)

            df[col] = sklearn.preprocessing.scale(df[col].values)

    df.dropna(inplace = True)

    sequential_data = []
    prev_days = deque( maxlen = SEQ_LEN)

    for i in df.values:
        prev_days.append(([n for n in i[:-1]])) #excluding target
        if len(prev_days) == SEQ_LEN:
            sequential_data.append([np.array(prev_days),i[-1]])

    random.shuffle(sequential_data)

    buys = []
    sells = []

    for seq, target in sequential_data:
        if target ==0:
            sells.append([seq,target])
        elif target == 1:
            buys.append([seq,target])

    random.shuffle(buys)
    random.shuffle(sells)

    lower =min(len(buys),len(sells))

    buys = buys[1:lower]
    sells = sells[1:lower]

    sequential_data = buys+ sells
    random.shuffle(sequential_data)

    X=[]
    y=[]
    for seq,target in sequential_data:
        X.append(seq)
        y.append(target)
    return np.array(X),y


def classify(current, future):
    if float(future) > float(current):
        return 1
    else:
        return 0

#we want to join all dataframes on the common index that is time.

main_df = pd.DataFrame()

ratios = ["BTC-USD","LTC-USD","ETH-USD","BCH-USD"]

for ratio in ratios:
    dataset = f"{ratio}.csv"

    df = pd.read_csv(dataset, names =["time","low","high","open","close","volume"])
    df.rename(columns = {"close": f"{ratio}_close","volume":f"{ratio}_volume"},inplace = True)
    df.set_index("time",inplace = True)
    df = df[[f"{ratio}_close",f"{ratio}_volume"]]
    if len(main_df)==0:
        main_df = df
    else:          #joining all the 4 datasets
        main_df = main_df.merge(df, left_index=True, right_index=True)

main_df['future'] = main_df[f"{RATIO_TO_PREDICT}_close"].shift(-FUTURE_PERIOD_PREDICT)

main_df['target'] = list(map(classify,main_df[f"{RATIO_TO_PREDICT}_close"],main_df['future']))

times = sorted(main_df.index.values)
last_5pct = times[-int(0.05*len(times))]

validation_main_df = main_df[(main_df.index>= last_5pct)]
main_df = main_df[(main_df.index<last_5pct)]

train_X, train_y = preprocess_df(main_df)
validation_X, validation_y = preprocess_df(validation_main_df)

print(f"train data: {len(train_X)}  validation : {len(validation_X)}")
print(f"Dont buys: {train_y.count(0)},  buys: {train_y.count(1)}")   #just checking that all are of same size.
print(f"validation dont buys: {validation_y.count(0)} validation buys : {validation_y.count(1)}")

model = Sequential()
model.add(CuDNNLSTM(128,input_shape =(train_X.shape[1:]),return_sequences = True))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(CuDNNLSTM(128,input_shape =(train_X.shape[1:]),return_sequences = True))
model.add(Dropout(0.1))
model.add(BatchNormalization())
model.add(CuDNNLSTM(128,input_shape =(train_X.shape[1:])))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Dense(32,activation='tanh'))
model.add(Dropout(0.2))

model.add(Dense(2,activation='softmax'))
opt = tf.keras.optimizers.Adam(lr=0.001,decay = 1e-6)

model.compile(loss='sparse_categorical_crossentropy',optimizer=opt,metrics=['accuracy'])

#tensorboard = TensorBoard(log_dir=f'logs/{NAME}')
#filepath = "RNN_Final-{epoch:02d}-{val_acc:.3f}"
#checkpoint = ModelCheckpoint("models/{}.model.format(filepath,monitor='val_acc', verbose =1,save_best_only=True,mode='max')

history = model.fit(train_X,train_y,batch_size=BATCH_SIZE,epochs=EPOCHS,validation_data=(validation_X,validation_y))



train data: 74620  validation : 4090
Dont buys: 37310,  buys: 37310
validation dont buys: 2045 validation buys : 2045
Train on 74620 samples, validate on 4090 samples
Epoch 1/9
74620/74620 [==============================] - 26s 343us/sample - loss: 0.7145 - acc: 0.5145 - val_loss: 0.6929 - val_acc: 0.5105
Epoch 2/9
74620/74620 [==============================] - 24s 327us/sample - loss: 0.6870 - acc: 0.5453 - val_loss: 0.6933 - val_acc: 0.5408
Epoch 3/9
74620/74620 [==============================] - 24s 327us/sample - loss: 0.6805 - acc: 0.5646 - val_loss: 0.6885 - val_acc: 0.5413
Epoch 4/9
74620/74620 [==============================] - 24s 325us/sample - loss: 0.6843 - acc: 0.5475 - val_loss: 0.6951 - val_acc: 0.5274
Epoch 5/9
74620/74620 [==============================] - 24s 328us/sample - loss: 0.6842 - acc: 0.5484 - val_loss: 0.6878 - val_acc: 0.5479
Epoch 6/9
74620/74620 [==============================] - 24s 324us/sample - loss: 0.6826 - acc: 0.5535 - val_loss: 0.7023 - val_acc: 